Lineaire blok codes

<u>Naam:</u> Ferre Van der Vieren (r0851455)

In deze korte notebook zullen we stap voor stap syndroomdecodering met lineaire blok codes implementeren. Gegeven een generatormatrix $F$ en een codewoord $c$, al dan niet met een fout, is het de bedoeling om het originele woord $w$ te vinden.

## Gauss eliminatie

Elke generatormatrix $F$ kan omgevormd worden tot een equivalente generatormatrix met getransponeerde in volledige rij-echolonvorm. Om de code in standaardvorm te brengen passen we Gauss-eliminatie toe op de getransponeerde van de generatormatrix. We implementeren hieronder de methode $\texttt{gauss(F, q)}$ die de generatormatrix $F$ omzet naar de equivalente code in standaardvorm. We werken steeds in het eindig veld $\mathbb{Z}_q$ met $q$ elementen. Een deling door een bepaald getal wordt dus vervangen door een vermeningvuldiging met het multiplicatief invers van dat getal. Hierom definiëren we $\texttt{inv(a, q)}$ die het inverse element van $a$ berekent $\mod q$. We gebruiken hier de stelling van Bezout en het uitgebreide Euclidische algoritme voor.

In [1]:
import numpy as np

def gcd(a, m):
    return eaa(a, m, False)

# using Extended Euclidean Algorithm to efficiently calculate gcd or Bezout's identity
# @return
    #  if (bezout == False): gcd(a, b)
    #  if (bezout == True): Bezout's identity coefficients s and t for which gcd(a, b) = s*a + t*b
def eaa(x, y, bezout = True):
    (a, new_a) = (x, y)
    (b, new_b) = (1, 0)
    (c, new_c) = (0, 1)
    
    while new_a != 0:
        q = a // new_a
        
        # coefficients
        (a, new_a) = (new_a, a - q * new_a)
        (b, new_b) = (new_b, b - q * new_b)
        (c, new_c) = (new_c, c - q * new_c)
        
    if (bezout):
        return (b, c)
    return a

def inv(a, q):
    if (gcd(a, q) != 1):  # if gcd(a, q) != 1, then a is not a unit
        return False
    b = eaa(a, q)
    return (b[0] % q)

def gauss(A, p):
    dim = A.shape
    
    if (dim[0] > dim[1]): # in this case F must be transposed (for this implementation to work)
        A = np.transpose(A)
        dim = A.shape

    for i in range(dim[0]):
        if A[i][i] != 0:
            inverse = inv(A[i][i], p)
            A[i] = np.dot(A[i], inverse)

            for j in range(dim[0]):
                if i == j:  
                    continue  # skip pivot row
                A[j] = np.add(A[j], np.dot(A[i], -A[j][i])) % p
    return A % p

assert(inv(5, 9) == 2)
assert(np.allclose(gauss(np.array([[2, 0], [1, 1]]), 3), np.array([[1, 0],[0, 1]])))
assert(np.allclose(gauss(np.array([[1, 1, 2], [2, 0, 1]]), 3), np.array([[1, 0 , 2],[0,1,0]])))

ModuleNotFoundError: No module named 'numpy'

## Syndroomdecoderingstabel

Voor de decodering hebben we de syndroomdecoderingstabel nodig. Deze geeft aan welk syndroom door welke fout veroorzaakt wordt. Hiervoor moet men eerst de pariteitsmatrix opstellen, die uit de standaardvorm van de generatormatrix gehaald kan worden. De methode $\texttt{parity_matrix(F, q)}$ berekent de pariteitsmatrix, gegeven een algemen generatormatrix $F$ over het veld $\mathbb{Z}_q$. De methode $\texttt{decoding_table(H, q)}$ geeft de syndroomdecoderingstabel weer in de vorm van een dictionary. De input $H$ is de pariteitsmatrix van een code.

In [2]:
import itertools

def parity_matrix(F, q):
    G = gauss(F, q)
    n = G.shape[0] 
    N = G.shape[1]
    P = (G[:, n:]*(-1) % q)
    
    id_matrix = np.identity(N-n, int)
    
    return np.vstack((P, id_matrix))

def g(y):  # weight of element y is equal to amount of non-zero elements
    non_zero = 0
    for el in y:
        if el != 0:
            non_zero += 1
    return non_zero

def decoding_table(H, q):
    N = H.shape[0]
    n = N - H.shape[1]
    H_t = np.transpose(H)
    table = {}
    
    lst = [i for i in range(q)]
    args = [lst for i in range(N)]
    for y in list(itertools.product(*args)):
        s = tuple(H_t.dot(y) % q)
        if s not in table:
            table[s] = list(y)
            continue
        if (g(y) < g(tuple(table[s]))):
            table[s] = list(y)
    return table

## Decodering

We implementeren tot slot de methode $\texttt{decode(c, F, p)}$ die een lijst met eventuele codewoorden $m$ decodeert in het veld $\mathbb{Z}_p$ gegeven de generatormatrix F, waarbij tijdens de transmissie al dan niet fouten zijn toegevoegd. 

In [3]:
def decode(c, F, p):
    lst = []
    H = parity_matrix(F, p)
    H_t = np.transpose(H)
    N = H.shape[0]
    n = N - H.shape[1]
    table = decoding_table(H, p)
    for y in c:
        s = tuple(H_t.dot(y) % p)
        a = [a_i - b_i for a_i, b_i in zip(y, table[s])]
        lst.append(a[0:n])
    return lst
    
F = np.array([[1,1,1,2,0,1],[2,0,2,1,2,1],[1,1,0,1,0,2]]).T
c = [[1, 0, 2, 0,  1, 1], [1, 2, 2, 0, 1, 1], [1,1,1,2,0,1]]
m = decode(c, F, 3)
assert(m == [[1, 0, 2], [1, 0, 2], [1, 1, 1]])

NameError: name 'np' is not defined